In [572]:
import pandas as pd

In [573]:
def float_or_false(value):
    try:
        if float(value):
            return True
    except ValueError:
        return False

In [574]:
def mod_false(df):
    for i in range(len(df)):
        if df.loc[i,'N-terminal_Modification'] != 'Free' or df.loc[i,'C-terminal_Modification'] != 'Free':
            df.loc[i,'is_mod'] = 'True'
            df.loc[i,'modifications'] = f"{df.loc[i, 'N-terminal_Modification']}, {df.loc[i, 'C-terminal_Modification']}"
        else:
            df.loc[i,'is_mod'] = 'False'
            df.loc[i,'modifications'] = ''

In [575]:
df = pd.read_csv('antiviral_original.csv')

In [576]:
df2 = df
mod_false(df2)
df2.drop(['N-terminal_Modification', 'C-terminal_Modification'], axis=1, inplace=True)
df2['modifications'] = df2['modifications'].replace([', Free', 'Free,'], '', regex=True)
df2 = df2.rename(columns={'Sequence': 'sequence'})
df2['sequence'] = df2['sequence'].str.upper()

In [577]:
df2

,sequence,Source,Target_Organism,Assay,Linear_Cyclic,Other_Modification,Binding_Target,experimental_characteristics,Half_Life,is_mod,modifications
0,AAAMSQVTN,Synthetic construct,HIV,Protease inhibition assay,Linear,NaN,autolysis,E_coli,>10 hour,False,
1,AAAMSQVTN,Synthetic construct,HIV,Protease inhibition assay,Linear,NaN,autolysis,Mammalian,4.4 hour,False,
2,AAAMSQVTN,Synthetic construct,HIV,Protease inhibition assay,Linear,NaN,autolysis,Yeast,>20 hour,False,
3,AACEVAKNLNESLIDLQELGKYEQYIKW,Synthetic construct(derived from SARS-CoV spik...,SARS-CoV,Plaque formation assay,Linear,NaN,membrane,E_coli,>10 hour,False,
4,AACEVAKNLNESLIDLQELGKYEQYIKW,Synthetic construct,SARS-CoV,Plaque reduction assay,Linear,NaN,cell membrane,Mammalian,4.4 hour,False,
...,...,...,...,...,...,...,...,...,...,...,...
5953,YAGAVVNDL,Synthetic construct,HSV,Ribonucleotide reductase assay,Linear,NaN,ribonucleotide reductase,Mammalian,NaN,True,Acetylation
5954,YAGAVVNDL,Synthetic construct,HSV,Ribonucleotide reductase assay,Linear,NaN,ribonucleotide reductase,E_coli,NaN,False,
5955,YAIIXYNKYXNC,Synthetic construct,"ZIKV,DENV,WNV",protease enzymatic inhibition assay,Linear,The 'X' at position 5 indicates N-methyl-4-O-m...,NS2B-NS3 protease,Yeast,NaN,True,Thioether-acyl moiety (−S-Ac−)(forms a macrocy...
5956,YAIIXYNKYXNC,Synthetic construct,"ZIKV,DENV,WNV",protease enzymatic inhibition assay,Linear,The 'X' at position 5 indicates N-methyl-4-O-m...,NS2B-NS3 protease,E_coli,NaN,True,Thioether-acyl moiety (−S-Ac−)(forms a macrocy...


In [578]:
#dimensiones de datos duplicados
df2[df2.duplicated(subset=['sequence', 'modifications'], keep=False)].shape

(5958, 11)

In [579]:
#dimensiones de datos no duplicados
df2[~df2.duplicated(subset=['sequence', 'modifications'], keep=False)].shape

(0, 11)

In [580]:
#separacion de datos duplicados que tengan la misma secuencia y modificaciones (se examinan las secuencia, si modificaiones y secuencia es igual pero half_life_seconds es diferente se borran todos los duplicados)
df_data_duplicated=(df2[df2.duplicated(subset=['sequence', 'modifications'], keep=False)]).sort_values(by='sequence')
df_data_duplicated

,sequence,Source,Target_Organism,Assay,Linear_Cyclic,Other_Modification,Binding_Target,experimental_characteristics,Half_Life,is_mod,modifications
0,AAAMSQVTN,Synthetic construct,HIV,Protease inhibition assay,Linear,NaN,autolysis,E_coli,>10 hour,False,
1,AAAMSQVTN,Synthetic construct,HIV,Protease inhibition assay,Linear,NaN,autolysis,Mammalian,4.4 hour,False,
2,AAAMSQVTN,Synthetic construct,HIV,Protease inhibition assay,Linear,NaN,autolysis,Yeast,>20 hour,False,
3,AACEVAKNLNESLIDLQELGKYEQYIKW,Synthetic construct(derived from SARS-CoV spik...,SARS-CoV,Plaque formation assay,Linear,NaN,membrane,E_coli,>10 hour,False,
4,AACEVAKNLNESLIDLQELGKYEQYIKW,Synthetic construct,SARS-CoV,Plaque reduction assay,Linear,NaN,cell membrane,Mammalian,4.4 hour,False,
...,...,...,...,...,...,...,...,...,...,...,...
5878,YXIAKYNXXIPC,Synthetic construct,"ZIKV,DENV,WNV",protease enzymatic inhibition assay,Linear,The 'X' at position 2 indicates N-methyl-glyci...,NS2B-NS3 protease,E_coli,2 min,True,Thioether-acyl moiety (−S-Ac−)(forms a macrocy...
5879,YXIAKYNXXIPC,Synthetic construct,"ZIKV,DENV,WNV",protease enzymatic inhibition assay,Linear,The 'X' at position 2 indicates N-methyl-glyci...,NS2B-NS3 protease,Yeast,10 min,True,Thioether-acyl moiety (−S-Ac−)(forms a macrocy...
5882,YXKXKXXKXXKC,Synthetic construct,"ZIKV,DENV,WNV",protease enzymatic inhibition assay,Linear,"The 'X' at position 2,7,9 and 10 indicates MeY...",NS2B-NS3 protease,Yeast,10 min,True,Thioether-acyl moiety (−S-Ac−)(forms a macrocy...
5880,YXKXKXXKXXKC,Synthetic construct,"ZIKV,DENV,WNV",protease enzymatic inhibition assay,Linear,"The 'X' at position 2,7,9 and 10 indicates MeY...",NS2B-NS3 protease,Mammalian,2.8 hour,True,Thioether-acyl moiety (−S-Ac−)(forms a macrocy...


In [581]:
#separacion de datos duplicados que tengan la misma secuencia, modificaciones y half_life_seconds (de esto solo se guarda una copia) 
df_data_duplicated_2=(df2[df2.duplicated(subset=['sequence', 'modifications', 'Half_Life'], keep=False)]).sort_values(by='sequence')
df_data_duplicated_2

,sequence,Source,Target_Organism,Assay,Linear_Cyclic,Other_Modification,Binding_Target,experimental_characteristics,Half_Life,is_mod,modifications
3,AACEVAKNLNESLIDLQELGKYEQYIKW,Synthetic construct(derived from SARS-CoV spik...,SARS-CoV,Plaque formation assay,Linear,NaN,membrane,E_coli,>10 hour,False,
4,AACEVAKNLNESLIDLQELGKYEQYIKW,Synthetic construct,SARS-CoV,Plaque reduction assay,Linear,NaN,cell membrane,Mammalian,4.4 hour,False,
5,AACEVAKNLNESLIDLQELGKYEQYIKW,Synthetic construct,SARS-CoV,Plaque reduction assay,Linear,NaN,cell membrane,Yeast,>20 hour,False,
6,AACEVAKNLNESLIDLQELGKYEQYIKW,Synthetic construct,SARS-CoV,Plaque reduction assay,Linear,NaN,cell membrane,E_coli,>10 hour,False,
7,AACEVAKNLNESLIDLQELGKYEQYIKW,Synthetic construct(derived from SARS-CoV spik...,SARS-CoV,Plaque formation assay,Linear,NaN,membrane,Mammalian,4.4 hour,False,
...,...,...,...,...,...,...,...,...,...,...,...
5652,YENQKQIANQFNKAISQIQESLTTTSTA,Synthetic construct,SARS-CoV PsV,HIV‐luc/SARS Pseudotyped Virus Entry Inhibitio...,Linear,NaN,not found,Yeast,10 min,False,
5650,YENQKQIANQFNKAISQIQESLTTTSTA,Synthetic construct,SARS-CoV PsV,HIV‐luc/SARS Pseudotyped Virus Entry Inhibitio...,Linear,NaN,not found,E_coli,2 min,False,
5649,YENQKQIANQFNKAISQIQESLTTTSTA,Synthetic construct,SARS-CoV PsV,HIV‐luc/SARS Pseudotyped Virus Entry Inhibitio...,Linear,NaN,not found,Mammalian,2.8 hour,False,
5654,YENQKQIANQFNKAISQIQESLTTTSTA,Synthetic construct(derived from SARS-CoV spik...,SARS-CoV,Luciferase assay,Linear,NaN,Not found,Mammalian,2.8 hour,False,


In [582]:
#separacion de datos no duplicados
df_not_duplicated=(df2[~df2.duplicated(subset=['sequence', 'modifications'], keep=False)]).sort_values(by='sequence')
#se concatenan los datos no duplicados con los datos duplicados pero que se mantiene una copia
#Como quito los datos duplicados que tiene casi todo igual
df_not_duplicated=pd.concat([df_not_duplicated, df_data_duplicated_2])
df_not_duplicated

,sequence,Source,Target_Organism,Assay,Linear_Cyclic,Other_Modification,Binding_Target,experimental_characteristics,Half_Life,is_mod,modifications
3,AACEVAKNLNESLIDLQELGKYEQYIKW,Synthetic construct(derived from SARS-CoV spik...,SARS-CoV,Plaque formation assay,Linear,NaN,membrane,E_coli,>10 hour,False,
4,AACEVAKNLNESLIDLQELGKYEQYIKW,Synthetic construct,SARS-CoV,Plaque reduction assay,Linear,NaN,cell membrane,Mammalian,4.4 hour,False,
5,AACEVAKNLNESLIDLQELGKYEQYIKW,Synthetic construct,SARS-CoV,Plaque reduction assay,Linear,NaN,cell membrane,Yeast,>20 hour,False,
6,AACEVAKNLNESLIDLQELGKYEQYIKW,Synthetic construct,SARS-CoV,Plaque reduction assay,Linear,NaN,cell membrane,E_coli,>10 hour,False,
7,AACEVAKNLNESLIDLQELGKYEQYIKW,Synthetic construct(derived from SARS-CoV spik...,SARS-CoV,Plaque formation assay,Linear,NaN,membrane,Mammalian,4.4 hour,False,
...,...,...,...,...,...,...,...,...,...,...,...
5652,YENQKQIANQFNKAISQIQESLTTTSTA,Synthetic construct,SARS-CoV PsV,HIV‐luc/SARS Pseudotyped Virus Entry Inhibitio...,Linear,NaN,not found,Yeast,10 min,False,
5650,YENQKQIANQFNKAISQIQESLTTTSTA,Synthetic construct,SARS-CoV PsV,HIV‐luc/SARS Pseudotyped Virus Entry Inhibitio...,Linear,NaN,not found,E_coli,2 min,False,
5649,YENQKQIANQFNKAISQIQESLTTTSTA,Synthetic construct,SARS-CoV PsV,HIV‐luc/SARS Pseudotyped Virus Entry Inhibitio...,Linear,NaN,not found,Mammalian,2.8 hour,False,
5654,YENQKQIANQFNKAISQIQESLTTTSTA,Synthetic construct(derived from SARS-CoV spik...,SARS-CoV,Luciferase assay,Linear,NaN,Not found,Mammalian,2.8 hour,False,


In [583]:
#se borran valores nulos y se eliminan los duplicados sobrantes de los que se guarda una copia y se ordenan los datos
df_not_duplicated.dropna(subset=['sequence', 'Half_Life'], inplace=True)
df_not_duplicated.drop_duplicates(subset=['sequence', 'modifications'], keep='first', inplace=True)
df_not_duplicated.sort_index(inplace=True)

In [584]:
df_not_duplicated

,sequence,Source,Target_Organism,Assay,Linear_Cyclic,Other_Modification,Binding_Target,experimental_characteristics,Half_Life,is_mod,modifications
3,AACEVAKNLNESLIDLQELGKYEQYIKW,Synthetic construct(derived from SARS-CoV spik...,SARS-CoV,Plaque formation assay,Linear,NaN,membrane,E_coli,>10 hour,False,
57,ACFPWGKEYCGGK,Synthetic construct,HCV,Surface plasmon resonance,Cyclic,Disulfide bond between Cys2 and Cys10,RNA-dependent RNA polymerase,E_coli,>10 hour,False,
71,ACFPWGNQWCGGK,Synthetic construct,HCV,Surface plasmon resonance,Cyclic,Disulfide bond between Cys2 and Cys10,RNA-dependent RNA polymerase,E_coli,>10 hour,False,
75,ACFPWGNTWCGGK,Synthetic construct,HCV,Surface plasmon resonance,Cyclic,Disulfide bond between Cys2 and Cys10,RNA-dependent RNA polymerase,Mammalian,4.4 hour,False,
404,ATAGWTFGAGAALQIPFAMQMAY,Synthetic construct,SARS-CoV,Plaque reduction assay,Linear,NaN,cell membrane,Mammalian,4.4 hour,False,
...,...,...,...,...,...,...,...,...,...,...,...
5265,WEDWVRWI,Synthetic construct(derived from the Membrane-...,FIV,enzyme-linked immunosorbent assay(ELISA),Linear,NaN,Not found,Yeast,3 min,False,
5376,WEQKIEELLKKAEEQQKKNEEELKKLEX,Synthetic construct,"HIV,SIV","cell-fusion assay,single-cycle infection assay",Linear,The 'X' at position 28 is Lys-C16(palmitic acid),membrane,E_coli,2 min,True,"Acetylation, Amidation"
5619,YAGAVVNDL,Synthetic construct,HSV,Ribonucleotide reductase assay,Linear,NaN,ribonucleotide reductase,Yeast,10 min,True,Acetylation
5653,YENQKQIANQFNKAISQIQESLTTTSTA,Synthetic construct(derived from SARS-CoV spik...,SARS-CoV,Luciferase assay,Linear,NaN,Not found,Yeast,10 min,False,


In [585]:
df_float=df_not_duplicated[~df_not_duplicated['Half_Life'].str.startswith('>', '<')]
df_object=df_not_duplicated[df_not_duplicated['Half_Life'].str.startswith('>', '<')]

In [586]:
df_float = df_float[df_float['Half_Life'].str.contains(r'^\d+\.?\d*\s*(hour|min)$')]
df_float.reset_index(drop=True, inplace=True)
for i in range(len(df_float)):
    if 'hour' in df_float.loc[i, 'Half_Life']:
        df_float.loc[i, 'Half_Life'] = float(df_float.loc[i, 'Half_Life'].split(' ')[0]) * 3600
    else:
        df_float.loc[i, 'Half_Life'] = float(df_float.loc[i, 'Half_Life'].split(' ')[0]) * 60

C:\Users\hantr\AppData\Local\Temp\ipykernel_10404\1304014241.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_float = df_float[df_float['Half_Life'].str.contains(r'^\d+\.?\d*\s*(hour|min)$')]


In [587]:
df_float.reset_index(drop=True, inplace=True)
df_object.reset_index(drop=True, inplace=True)

In [588]:
df_float

,sequence,Source,Target_Organism,Assay,Linear_Cyclic,Other_Modification,Binding_Target,experimental_characteristics,Half_Life,is_mod,modifications
0,ACFPWGNTWCGGK,Synthetic construct,HCV,Surface plasmon resonance,Cyclic,Disulfide bond between Cys2 and Cys10,RNA-dependent RNA polymerase,Mammalian,15840.0,False,
1,ATAGWTFGAGAALQIPFAMQMAY,Synthetic construct,SARS-CoV,Plaque reduction assay,Linear,NaN,cell membrane,Mammalian,15840.0,False,
2,DLSLDFEKLNVTLLDLTYEMNRIQDAIKKLNESYINLKE,Synthetic construct,"SARS-CoV, MHV",Infection inhibition assay,Linear,NaN,membrane,Yeast,180.0,False,
3,DVDLGDISGINASVVNIQKEIDRLNEVAKNLNESLIDLQELGKYEQYI,Synthetic construct(derived from SARS-CoV spik...,SARS-CoV,Luciferase assay,Linear,NaN,Not found,Mammalian,3960.0,False,
4,DWVRWI,Synthetic construct(derived from the Membrane-...,FIV,enzyme-linked immunosorbent assay(ELISA),Linear,NaN,Not found,Yeast,180.0,False,
...,...,...,...,...,...,...,...,...,...,...,...
141,WEDWVRWI,Synthetic construct(derived from the Membrane-...,FIV,enzyme-linked immunosorbent assay(ELISA),Linear,NaN,Not found,Yeast,180.0,False,
142,WEQKIEELLKKAEEQQKKNEEELKKLEX,Synthetic construct,"HIV,SIV","cell-fusion assay,single-cycle infection assay",Linear,The 'X' at position 28 is Lys-C16(palmitic acid),membrane,E_coli,120.0,True,"Acetylation, Amidation"
143,YAGAVVNDL,Synthetic construct,HSV,Ribonucleotide reductase assay,Linear,NaN,ribonucleotide reductase,Yeast,600.0,True,Acetylation
144,YENQKQIANQFNKAISQIQESLTTTSTA,Synthetic construct(derived from SARS-CoV spik...,SARS-CoV,Luciferase assay,Linear,NaN,Not found,Yeast,600.0,False,


In [589]:
df_not_duplicated

,sequence,Source,Target_Organism,Assay,Linear_Cyclic,Other_Modification,Binding_Target,experimental_characteristics,Half_Life,is_mod,modifications
3,AACEVAKNLNESLIDLQELGKYEQYIKW,Synthetic construct(derived from SARS-CoV spik...,SARS-CoV,Plaque formation assay,Linear,NaN,membrane,E_coli,>10 hour,False,
57,ACFPWGKEYCGGK,Synthetic construct,HCV,Surface plasmon resonance,Cyclic,Disulfide bond between Cys2 and Cys10,RNA-dependent RNA polymerase,E_coli,>10 hour,False,
71,ACFPWGNQWCGGK,Synthetic construct,HCV,Surface plasmon resonance,Cyclic,Disulfide bond between Cys2 and Cys10,RNA-dependent RNA polymerase,E_coli,>10 hour,False,
75,ACFPWGNTWCGGK,Synthetic construct,HCV,Surface plasmon resonance,Cyclic,Disulfide bond between Cys2 and Cys10,RNA-dependent RNA polymerase,Mammalian,4.4 hour,False,
404,ATAGWTFGAGAALQIPFAMQMAY,Synthetic construct,SARS-CoV,Plaque reduction assay,Linear,NaN,cell membrane,Mammalian,4.4 hour,False,
...,...,...,...,...,...,...,...,...,...,...,...
5265,WEDWVRWI,Synthetic construct(derived from the Membrane-...,FIV,enzyme-linked immunosorbent assay(ELISA),Linear,NaN,Not found,Yeast,3 min,False,
5376,WEQKIEELLKKAEEQQKKNEEELKKLEX,Synthetic construct,"HIV,SIV","cell-fusion assay,single-cycle infection assay",Linear,The 'X' at position 28 is Lys-C16(palmitic acid),membrane,E_coli,2 min,True,"Acetylation, Amidation"
5619,YAGAVVNDL,Synthetic construct,HSV,Ribonucleotide reductase assay,Linear,NaN,ribonucleotide reductase,Yeast,10 min,True,Acetylation
5653,YENQKQIANQFNKAISQIQESLTTTSTA,Synthetic construct(derived from SARS-CoV spik...,SARS-CoV,Luciferase assay,Linear,NaN,Not found,Yeast,10 min,False,


In [590]:
df_float.rename(columns={'Half_Life': 'half_life_seconds'}, inplace=True)
df_object.rename(columns={'Half_Life': 'half_life'}, inplace=True)

C:\Users\hantr\AppData\Local\Temp\ipykernel_10404\905327210.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_object.rename(columns={'Half_Life': 'half_life'}, inplace=True)


In [591]:
df_float=df_float.filter(['sequence', 'half_life_seconds', 'is_mod', 'modifications', 'experimental_characteristics'])
df_object=df_object.filter(['sequence', 'half_life', 'is_mod', 'modifications', 'experimental_characteristics'])

In [592]:
df_float

,sequence,half_life_seconds,is_mod,modifications,experimental_characteristics
0,ACFPWGNTWCGGK,15840.0,False,,Mammalian
1,ATAGWTFGAGAALQIPFAMQMAY,15840.0,False,,Mammalian
2,DLSLDFEKLNVTLLDLTYEMNRIQDAIKKLNESYINLKE,180.0,False,,Yeast
3,DVDLGDISGINASVVNIQKEIDRLNEVAKNLNESLIDLQELGKYEQYI,3960.0,False,,Mammalian
4,DWVRWI,180.0,False,,Yeast
...,...,...,...,...,...
141,WEDWVRWI,180.0,False,,Yeast
142,WEQKIEELLKKAEEQQKKNEEELKKLEX,120.0,True,"Acetylation, Amidation",E_coli
143,YAGAVVNDL,600.0,True,Acetylation,Yeast
144,YENQKQIANQFNKAISQIQESLTTTSTA,600.0,False,,Yeast


In [593]:
df_object

,sequence,half_life,is_mod,modifications,experimental_characteristics
0,AACEVAKNLNESLIDLQELGKYEQYIKW,>10 hour,False,,E_coli
1,ACFPWGKEYCGGK,>10 hour,False,,E_coli
2,ACFPWGNQWCGGK,>10 hour,False,,E_coli
3,AWDFGSLGGVFTSIGKALHQVFGAIYGAA,>20 hour,False,,Yeast
4,DISGINASVVNIQKEIDRLNEVAKNLNESLIDLQELGKYE,>10 hour,False,,E_coli
5,EAIIRILQQLLFIHFRIGRRRRRRRR,>10 hour,True,"Acetylation, Amidation",E_coli
6,ELDSFKEELDKYFKNHTSPDVDLGDISGINASVVNIQKEIDRLNEV...,>10 hour,False,,E_coli
7,EWDREINNYTSLIHSLIEESQXQQEKNEQELLELDKWASLW,>10 hour,False,,E_coli
8,EWDREINNYTSLIHSLIEEXQNQQEKNEQELLELDKWASLW,>10 hour,False,,E_coli
9,EWDREINNYTSLIHSLIXESQNQQEKNEQELLELDKWASLW,>10 hour,False,,E_coli


In [594]:
df_float.to_csv('antiviral.csv', index=False)
df_object.to_csv('antiviral_object.csv', index=False)